In [2]:
from tensorflow import keras
from sklearn.metrics import accuracy_score, mean_squared_error, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from tensorflow.keras.datasets import fashion_mnist

# Load & Preprocess Data

In [3]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [4]:
x_train = x_train / 255.0
x_test = x_test / 255.0

In [5]:
print(f'''
    Input and Output Shapes
    x_train :: {x_train.shape}
    x_test  :: {x_test.shape}
    y_train :: {y_train.shape}
    y_test  :: {y_test.shape}
''')


    Input and Output Shapes
    x_train :: (60000, 28, 28)
    x_test  :: (10000, 28, 28)
    y_train :: (60000,)
    y_test  :: (10000,)

